# Settings

In [1]:
import numpy as np
import torch
import os
from module_space import *

if not os.path.exists('regrets'):
    os.mkdir('regrets')

### Bandit settings

In [2]:
T = 2000
n_sim = 10 # number of simulations


n_arms = 12 # N
n_features_default = 20 # d
n_assortment = 4 # K
n_samples = 50 # M, number of samples per each round and arm, for TS

noise_std = 0.01 # noise of reward: xi = noise_std*N(0,1)


def F(x): # round_reward_function
    if len(np.shape(x)) == 1: # if x is a vector
        return np.sum(x)
    else: # if x is a matrix
        return np.sum(x, axis=1)                

### Parameter settings

In [160]:
reg_factor = 1 # lambda
delta = 0.1 # delta
exploration_variance = 0.8 # nu, for TS and CombLinUCB
confidence_scaling_factor = 1 # gamma, for CN-UCB, gamma is larger than nu

### Neural network settings

In [161]:
hidden_size_default = 20 # m
epochs = 100 # repeat training for each period
training_period = 5 ### training period
training_window = 100
learning_rate = 0.01

p = 0.0 # no dropout

use_cuda = False

###
#if torch.cuda.is_available():
#    use_cuda = True
#else:
#    use_cuda = False

# Experiment descirption
### Hidden functions
- Linear: $h_{1}(\mathbf{x}_{t,i}) = \mathbf{x}_{t,i}^{\top}\mathbf{a}$
- Quadratic: $h_{2}(\mathbf{x}_{t,i}) = (\mathbf{x}_{t,i}^{\top}\mathbf{a})^{2}$
- Non-linear: $h_{3}(\mathbf{x}_{t,i}) = \cos(\pi \mathbf{x}_{t,i}^{\top}\mathbf{a})$
- where $\mathbf{a} \sim U(-1,1)/\|\mathbf{a}\|_{2}$

### For each hidden function, compare the following algorithms
- CombLinUCB
- CombLinTS
- CN-UCB
- CN-TS(1): single reward sample
- CN-TS(30): optimistic sampling, sample size = 30 (default sample size is 1)

### Ablation study of feature dimension *d* and neural network width *m*
- Default value: $d = 20, m = 20$
- $d = \{20, 40\}$  for all algorithms

# Experiment settings

### Hidden function, bandit, learning algorithm and regret settings

In [162]:
h1 = "h1"
h2 = "h2"
h3 = "h3"

In [163]:
def experiment(lin_neural, ucb_ts, h_str, n_features=20, hidden_size=100, n_samples=1, save = ''):
    """ kind explanation
    """
    
    ## Hidden function
    SEED = 1234
    np.random.seed(SEED)

    a = np.random.uniform(low=-1.0, high=1.0, size=n_features)
    a /= np.linalg.norm(a, ord=2)
    
    if h_str == "h1":
        h = lambda x: np.dot(x, a)
        ###h = lambda x: 100*np.dot(x, a)
    elif h_str == "h2":
        h = lambda x: np.dot(x, a)**2
        ###h = lambda x: 100*np.dot(x, a)**2
    elif h_str == "h3":
        h = lambda x: np.cos(np.pi*np.dot(x, a))
        ###h = lambda x: 100*np.cos(np.pi*np.dot(x, a))
    
    ## Bandit
    bandit = ContextualBandit(T,
                              n_arms,
                              n_features, 
                              h,
                              noise_std=noise_std,
                              n_assortment=n_assortment,
                              n_samples=n_samples,
                              round_reward_function=F
                             )
    
    ## Learning algorithm and regret
    regrets = np.empty((n_sim, T))
    
    for i in range(n_sim):
        bandit.reset_rewards()
        
        if lin_neural == Neural:
            model = lin_neural(ucb_ts,
                               bandit,
                               hidden_size=hidden_size,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variance,
                               p=p,
                               training_window=training_window,
                               learning_rate=learning_rate,
                               epochs=epochs,
                               training_period=training_period,
                               use_cuda=use_cuda
                              )
            
            model.set_init_param(model.model.parameters()) # keep initial parameters for regularization
            
        if lin_neural == Lin:
            model = lin_neural(ucb_ts,
                               bandit,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variance
                              )
        
        model.run()
        regrets[i] = np.cumsum(model.regrets)
    
    if save: # save regrets
        np.save('regrets/' + save, regrets)    

# Experiment A: (h, d, m) - Algorithm - Regret

### (h1, 40, 20)

In [155]:
n_sim = 1
T = 2000

In [156]:
experiment(Neural, "UCB", h1, n_features=40, hidden_size=20, save='reg_h1_CNUCB_80_40')

  0%|          | 0/2000 [00:00<?, ?it/s, % optimal set of arms=0, total regret=0]


KeyboardInterrupt: 

In [126]:
experiment(Neural, "TS", h1, n_features=40, hidden_size=40, save='reg_h1_CNTS_80_40')

 44%|████▍     | 220/500 [00:23<00:29,  9.47it/s, total regret=14.3, % optimal set of arms=20.45%]


KeyboardInterrupt: 

In [ ]:
experiment(Neural, "TS", h1, n_features=80, hidden_size=40, n_samples=50, save='reg_h1_CNTSOpt_40_20')

### (h2, 40, 20)

In [167]:
experiment(Neural, "UCB", h2, n_features=40, hidden_size=20, save='reg_h2_CNUCB_40_20')

100%|██████████| 2000/2000 [01:31<00:00, 21.76it/s, total regret=61.2, % optimal set of arms=7.00%]


In [168]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=20, save='reg_h2_CNTS_40_20')

100%|██████████| 2000/2000 [01:27<00:00, 22.83it/s, total regret=49.6, % optimal set of arms=6.35%]


In [169]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=20, n_samples=50, save='reg_h2_CNTSOpt_40_20')

 54%|█████▎    | 1070/2000 [00:46<00:40, 22.86it/s, total regret=30.8, % optimal set of arms=6.64%]


KeyboardInterrupt: 

### (h3, 40, 20)

In [159]:
experiment(Neural, "UCB", h3, n_features=40, hidden_size=20, save='reg_h3_CNUCB_40_20')

100%|██████████| 2000/2000 [01:35<00:00, 20.89it/s, total regret=291, % optimal set of arms=0.20%]


In [164]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=20, save='reg_h3_CNTS_40_20')

100%|██████████| 2000/2000 [01:35<00:00, 21.02it/s, total regret=322, % optimal set of arms=0.20%]


In [166]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=20, n_samples=50, save='reg_h3_CNTSOpt_40_20')

100%|██████████| 2000/2000 [01:31<00:00, 21.86it/s, total regret=283, % optimal set of arms=0.15%]


### (h1, 20, 20)

In [54]:
T = 2000

In [55]:
experiment(Neural, "UCB", h1, n_features=10, hidden_size=20, save='reg_h1_CNUCB_10_20')

100%|██████████| 2000/2000 [01:11<00:00, 28.03it/s, total regret=16.8, % optimal set of arms=59.35%]


In [ ]:
experiment(Neural, "TS", h1, n_features=10, hidden_size=20, save='reg_h1_CNTS_10_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=10, hidden_size=20, n_samples=50, save='reg_h1_CNTSOpt_10_20')

### (h2, 20, 20)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=20, save='reg_h2_CNUCB_20_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=20, save='reg_h2_CNTS_20_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=20, n_samples=50, save='reg_h2_CNTSOpt_20_20')

### (h3, 20, 20)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=20, save='reg_h3_CNUCB_20_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=20, save='reg_h3_CNTS_20_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=20, n_samples=50, save='reg_h3_CNTSOpt_20_20')

### CombLinUCB

In [ ]:
experiment(Lin, "UCB", h1, n_features=20, save='reg_h1_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h2, n_features=20, save='reg_h2_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h3, n_features=20, save='reg_h3_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h1, n_features=40, save='reg_h1_CombLinUCB_40')

In [ ]:
experiment(Lin, "UCB", h2, n_features=40, save='reg_h2_CombLinUCB_40')

In [ ]:
experiment(Lin, "UCB", h3, n_features=40, save='reg_h3_CombLinUCB_40')

### CombLinTS

In [ ]:
experiment(Lin, "TS", h1, n_features=20, save='reg_h1_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h2, n_features=20, save='reg_h2_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h3, n_features=20, save='reg_h3_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h1, n_features=40, save='reg_h1_CombLinTS_40')

In [ ]:
experiment(Lin, "TS", h2, n_features=40, save='reg_h2_CombLinTS_40')

In [ ]:
experiment(Lin, "TS", h3, n_features=40, save='reg_h3_CombLinTS_40')

---------------------------------------

### (h1, 80, 20)

In [ ]:
experiment(Neural, "UCB", h1, n_features=80, hidden_size=20, save='reg_h1_CNUCB_80_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=80, hidden_size=20, save='reg_h1_CNTS_80_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=80, hidden_size=20, n_samples=50, save='reg_h1_CNTSOpt_80_20')

### (h2, 80, 20)

In [ ]:
experiment(Neural, "UCB", h2, n_features=80, hidden_size=20, save='reg_h2_CNUCB_80_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=80, hidden_size=20, save='reg_h2_CNTS_80_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=80, hidden_size=20, n_samples=50, save='reg_h2_CNTSOpt_80_20')

### (h3, 80, 20)

In [ ]:
experiment(Neural, "UCB", h3, n_features=80, hidden_size=20, save='reg_h3_CNUCB_80_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=80, hidden_size=20, save='reg_h3_CNTS_80_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=80, hidden_size=20, save='reg_h3_CNTSOpt_80_20')

-------------------------

### (h1, 40, 20)

In [ ]:
experiment(Neural, "UCB", h1, n_features=40, hidden_size=20, save='reg_h1_CNUCB_40_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=40, hidden_size=20, save='reg_h1_CNTS_40_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=40, hidden_size=20, n_samples=50, save='reg_h1_CNTSOpt_40_20')

### (h2, 40, 20)

In [ ]:
experiment(Neural, "UCB", h2, n_features=40, hidden_size=20, save='reg_h2_CNUCB_40_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=20, save='reg_h2_CNTS_40_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=20, n_samples=50, save='reg_h2_CNTSOpt_40_20')

### (h3, 40, 20)

In [ ]:
experiment(Neural, "UCB", h3, n_features=40, hidden_size=20, save='reg_h3_CNUCB_40_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=20, save='reg_h3_CNTS_40_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=20, n_samples=50, save='reg_h3_CNTSOpt_40_20')

### (h1, 20, 60)

In [ ]:
experiment(Neural, "UCB", h1, n_features=20, hidden_size=60, save='reg_h1_CNUCB_20_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=60, save='reg_h1_CNTS_20_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=60, n_samples=50, save='reg_h1_CNTSOpt_20_60')

### (h2, 20, 60)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=60, save='reg_h2_CNUCB_20_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=60, save='reg_h2_CNTS_20_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=60, n_samples=50, save='reg_h2_CNTSOpt_20_60')

### (h3, 20, 60)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=60, save='reg_h3_CNUCB_20_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=60, save='reg_h3_CNTS_20_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=60, n_samples=50, save='reg_h3_CNTSOpt_20_60')

### (h1, 60, 60)

In [ ]:
experiment(Neural, "UCB", h1, n_features=60, hidden_size=60, save='reg_h1_CNUCB_60_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=60, hidden_size=60, save='reg_h1_CNTS_60_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=60, hidden_size=60, n_samples=50, save='reg_h1_CNTSOpt_60_60')

### (h2, 60, 60)

In [ ]:
experiment(Neural, "UCB", h2, n_features=60, hidden_size=60, save='reg_h2_CNUCB_60_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=60, hidden_size=60, save='reg_h2_CNTS_60_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=60, hidden_size=60, n_samples=50, save='reg_h2_CNTSOpt_60_60')

### (h3, 60, 60)

In [ ]:
experiment(Neural, "UCB", h3, n_features=60, hidden_size=60, save='reg_h3_CNUCB_60_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=60, hidden_size=60, save='reg_h3_CNTS_60_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=60, hidden_size=60, n_samples=50, save='reg_h3_CNTSOpt_60_60')

### CombLinUCB

In [ ]:
experiment(Lin, "UCB", h1, n_features=20, save='reg_h1_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h2, n_features=20, save='reg_h2_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h3, n_features=20, save='reg_h3_CombLinUCB_20')

In [ ]:
experiment(Lin, "UCB", h1, n_features=40, save='reg_h1_CombLinUCB_40')

In [ ]:
experiment(Lin, "UCB", h2, n_features=40, save='reg_h2_CombLinUCB_40')

In [ ]:
experiment(Lin, "UCB", h3, n_features=40, save='reg_h3_CombLinUCB_40')

### CombLinTS

In [ ]:
experiment(Lin, "TS", h1, n_features=20, save='reg_h1_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h2, n_features=20, save='reg_h2_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h3, n_features=20, save='reg_h3_CombLinTS_20')

In [ ]:
experiment(Lin, "TS", h1, n_features=40, save='reg_h1_CombLinTS_40')

In [ ]:
experiment(Lin, "TS", h2, n_features=40, save='reg_h2_CombLinTS_40')

In [ ]:
experiment(Lin, "TS", h3, n_features=40, save='reg_h3_CombLinTS_40')

# Experiment B: Ablation study

In [ ]:
T = 500

### (h1, 20, 20)

In [ ]:
experiment(Neural, "UCB", h1, n_features=20, hidden_size=20, save='reg_h1_CNUCB_20_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=20, save='reg_h1_CNTS_20_20')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=20, n_samples=50, save='reg_h1_CNTSOpt_20_20')

### (h2, 20, 20)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=20, save='reg_h2_CNUCB_20_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=20, save='reg_h2_CNTS_20_20')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=20, n_samples=50, save='reg_h2_CNTSOpt_20_20')

### (h3, 20, 20)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=20, save='reg_h3_CNUCB_20_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=20, save='reg_h3_CNTS_20_20')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=20, n_samples=50, save='reg_h3_CNTSOpt_20_20')

### (h1, 20, 40)

In [ ]:
experiment(Neural, "UCB", h1, n_features=20, hidden_size=40, save='reg_h1_CNUCB_20_40')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=40, save='reg_h1_CNTS_20_40')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=40, n_samples=50, save='reg_h1_CNTSOpt_20_40')

### (h2, 20, 40)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=40, save='reg_h2_CNUCB_20_40')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=40, save='reg_h2_CNTS_20_40')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=40, n_samples=50, save='reg_h2_CNTSOpt_20_40')

### (h3, 20, 40)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=40, save='reg_h3_CNUCB_20_40')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=40, save='reg_h3_CNTS_20_40')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=40, n_samples=50, save='reg_h3_CNTSOpt_20_40')

### (h1, 20, 80)

In [ ]:
experiment(Neural, "UCB", h1, n_features=20, hidden_size=80, save='reg_h1_CNUCB_20_80')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=80, save='reg_h1_CNTS_20_80')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=80, n_samples=50, save='reg_h1_CNTSOpt_20_80')

### (h2, 20, 80)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=80, save='reg_h2_CNUCB_20_80')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=80, save='reg_h2_CNTS_20_80')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=80, n_samples=50, save='reg_h2_CNTSOpt_20_80')

### (h3, 20, 80)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=80, save='reg_h3_CNUCB_20_80')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=80, save='reg_h3_CNTS_20_80')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=80, n_samples=50, save='reg_h3_CNTSOpt_20_80')

### (h1, 20, 100)

In [ ]:
experiment(Neural, "UCB", h1, n_features=20, hidden_size=100, save='reg_h1_CNUCB_20_100')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=100, save='reg_h1_CNTS_20_100')

In [ ]:
experiment(Neural, "TS", h1, n_features=20, hidden_size=100, n_samples=50, save='reg_h1_CNTSOpt_20_100')

### (h2, 20, 100)

In [ ]:
experiment(Neural, "UCB", h2, n_features=20, hidden_size=100, save='reg_h2_CNUCB_20_100')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=100, save='reg_h2_CNTS_20_100')

In [ ]:
experiment(Neural, "TS", h2, n_features=20, hidden_size=100, n_samples=50, save='reg_h2_CNTSOpt_20_100')

### (h3, 20, 100)

In [ ]:
experiment(Neural, "UCB", h3, n_features=20, hidden_size=100, save='reg_h3_CNUCB_20_100')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=100, save='reg_h3_CNTS_20_100')

In [ ]:
experiment(Neural, "TS", h3, n_features=20, hidden_size=100, n_samples=50, save='reg_h3_CNTSOpt_20_100')

### (h1, 40, 60)

In [ ]:
experiment(Neural, "UCB", h1, n_features=40, hidden_size=60, save='reg_h1_CNUCB_40_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=40, hidden_size=60, save='reg_h1_CNTS_40_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=40, hidden_size=60, n_samples=50, save='reg_h1_CNTSOpt_40_60')

### (h2, 40, 60)

In [ ]:
experiment(Neural, "UCB", h2, n_features=40, hidden_size=60, save='reg_h2_CNUCB_40_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=60, save='reg_h2_CNTS_40_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=40, hidden_size=60, n_samples=50, save='reg_h2_CNTSOpt_40_60')

### (h3, 40, 60)

In [ ]:
experiment(Neural, "UCB", h3, n_features=40, hidden_size=60, save='reg_h3_CNUCB_40_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=60, save='reg_h3_CNTS_40_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=40, hidden_size=60, n_samples=50, save='reg_h3_CNTSOpt_40_60')

### (h1, 80, 60)

In [ ]:
experiment(Neural, "UCB", h1, n_features=80, hidden_size=60, save='reg_h1_CNUCB_80_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=80, hidden_size=60, save='reg_h1_CNTS_80_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=80, hidden_size=60, n_samples=50, save='reg_h1_CNTSOpt_80_60')

### (h2, 80, 60)

In [ ]:
experiment(Neural, "UCB", h2, n_features=80, hidden_size=60, save='reg_h2_CNUCB_80_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=80, hidden_size=60, save='reg_h2_CNTS_80_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=80, hidden_size=60, n_samples=50, save='reg_h2_CNTSOpt_80_60')

### (h3, 80, 60)

In [ ]:
experiment(Neural, "UCB", h3, n_features=80, hidden_size=60, save='reg_h3_CNUCB_80_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=80, hidden_size=60, save='reg_h3_CNTS_80_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=80, hidden_size=60, n_samples=50, save='reg_h3_CNTSOpt_80_60')

### (h1, 100, 60)

In [ ]:
experiment(Neural, "UCB", h1, n_features=100, hidden_size=60, save='reg_h1_CNUCB_100_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=100, hidden_size=60, save='reg_h1_CNTS_100_60')

In [ ]:
experiment(Neural, "TS", h1, n_features=100, hidden_size=60, n_samples=50, save='reg_h1_CNTSOpt_100_60')

### (h2, 100, 60)

In [ ]:
experiment(Neural, "UCB", h2, n_features=100, hidden_size=60, save='reg_h2_CNUCB_100_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=100, hidden_size=60, save='reg_h2_CNTS_100_60')

In [ ]:
experiment(Neural, "TS", h2, n_features=100, hidden_size=60, n_samples=50, save='reg_h2_CNTSOpt_100_60')

### (h3, 100, 60)

In [ ]:
experiment(Neural, "UCB", h3, n_features=100, hidden_size=60, save='reg_h3_CNUCB_100_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=100, hidden_size=60, save='reg_h3_CNTS_100_60')

In [ ]:
experiment(Neural, "TS", h3, n_features=100, hidden_size=60, n_samples=50, save='reg_h3_CNTSOpt_100_60')